# Improved Random Forest Model

### Three Approaches to Making a Better ML Model

1. More high-quality data.
2. Hyperparameter tuning of algorithm.
3. Trying different algorithm.

## Starting from Previous Model